# Taller de Física Computacional

Carlos Ruestes / Cristián Sánchez - Taller de Física Computacional - FCEN - UNCUYO

# Sesión 14: Modos normales de una cadena unidimensional de Lennard-Jones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
import scipy.optimize as opt

La cadena es la misma que la de la sesión anterior, definimos el potencial entre partículas y sus derivadas primera y segunda:

In [ ]:
EPSILON = 10.0
SIGMA = 2.5

In [ ]:
def vlj(r):
    return EPSILON*((SIGMA / r)**12 - 2*(SIGMA / r)**6) 

In [ ]:
def d_vlj(r):
    return EPSILON*( 12*SIGMA**6*r**(-7) -12*SIGMA**12*r**(-13) )

In [ ]:
def dd_vlj(r):
    return EPSILON*(-84*SIGMA**6*r**(-8)+156*SIGMA**12*r**(-14))

Definimos un número total de partículas y generamos una condición inicial para la optimización:

In [ ]:
NPART = 50
D_EQ = 2.431934945735542

In [ ]:
xcoords = np.linspace(0.0,D_EQ*(NPART-1),num=NPART) + np.random.rand((NPART))*0.01

La función para calcular las distancias en la recta real y sus derivadas primera y segunda:

In [ ]:
def rij(xi,xj):
    return abs(xj - xi)

In [ ]:
def d_rij(xi,xj,k):
    if (xi - xj) > 0.0 and k == 1:
        return 1.0
    elif (xi - xj) < 0.0 and k == 1:
        return -1.0
    elif (xi - xj) > 0.0 and k == 2:
        return -1.0
    elif (xi - xj) < 0.0 and k == 2:
        return  1.0
    else:
        raise ValueError('k sólo puede ser 1 o 2.')

In [ ]:
def dd_rij(xi,xj,k,l):
    return 0.0

En este caso, para obtener la geometría de equilibrio y los modos normales conectamos la cadena a dos puntos fijos con un resorte de alta constante de fuerza:

In [ ]:
K = 1000

Definimos funciones que calculan la energía total, su derivada respecto a la coordenada de una partícula y el vector gradiente:

In [ ]:
def energy(xs,L):
    energy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                energy += vlj(rij(xs[i],xs[j]))
    energy += 0.5 * K * (xs[0] - 0)**2
    energy += 0.5 * K * (xs[NPART-1] - L)**2
    return 0.5 * energy

In [ ]:
def d_energy(xs,k,L):
    denergy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                if k == i:
                    denergy += d_vlj(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],1)
                elif k == j:
                    denergy += d_vlj(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],2)   
    if (k == 0):
        denergy += K * (xs[0] - 0)
    if (k == NPART-1):
        denergy += K * (xs[NPART-1] - L)
    return denergy

In [ ]:
def grad_energy(xs,L):
    gradient = np.zeros((NPART))
    for i in range(0,NPART):
        gradient[i] = d_energy(xs,i,L)
    return gradient

Optimizamos la geometría de la cadena para un largo en el que esté en tensión:

In [ ]:
L = 122.16239314681823 * 1.006

In [ ]:
res = opt.minimize(energy, xcoords, method='BFGS', tol = 1e-6, jac=grad_energy, args=(L,),
                   options={'disp': True})
res.success

In [ ]:
equilibrio = res.x

In [ ]:
print(equilibrio[0],equilibrio[NPART-1])

Las siguientes son las fuerzas ejercidas por el resorte en las partículas de los extremos:

In [ ]:
- K * equilibrio[0]

In [ ]:
- K * (equilibrio[NPART-1] - L)

In [ ]:
largo = equilibrio[NPART-1] - equilibrio[0]
print("El largo de la cadena es ",largo)

In [ ]:
plt.plot(grad_energy(equilibrio,L))

In [ ]:
y = np.zeros_like(equilibrio) + 0.0
plt.figure(figsize=(10.0,1.0))
plt.plot(equilibrio,y,marker = "o")

In [ ]:
xcoords = equilibrio

La siguiente celda define una función que calcula la derivada segunda de la energía respecto a cada par de coordenadas

In [ ]:
def dd_energy(xs,k,l):
    ddenergy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                if (k == i) and (l == i):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],1,1)
                elif (k == i) and (l == j):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],2,1)
                elif (k == j) and (l == i):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],1,2)
                elif (k == j) and (l == j):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],2,2)
    if (k==0) and (l==0):
        ddenergy += K
    elif (k==NPART-1) and (l==NPART-1):
        ddenergy += K
    return ddenergy

## Cálculo de modos normales

Si nos restringimos a movimientos pequeños de las partículas respecto a su posición de equilibrio podemos escribir la energía potencial del sistema de la forma

$$ 2 V(q_1,\ldots,q_n) = \sum_{i,j=1}^N f_{ij} q_i q_j $$

donde $q_i = \Delta x_i = (x_i - x_i^0)$ y los $x_i^0$ son las posiciones en la geometría de equilibrio

$$ f_{ij} = \frac{\partial V}{\partial x_i \partial x_j} $$

A la expresión se llega haciendo una expansión en serie de la energía total en términos de los desplazamientos de las partículas respecto a su posición de equilibrio, y poniendo el cero de energía en la energía de equilibrio. Los términos que contienen a las derivadas primeras en la serie de Taylorr se anulan en la geometría de equilibrio.

Si las masas se consideran unitarias la energía cinética toma la forma:

$$ 2 T = \sum_{i=1}^N \dot{q}_i$$

Las ecuaciones de movimiento de los desplazamientos $q_j$ pueden escribirse de la siguiente forma:

$$ \frac{d}{dt}\frac{\partial T}{\partial \dot{q}_j} - \frac{\partial V}{\partial q_j} = 0 $$

Sustituyendo las expresiones para las energías cinética y potencial tenemos el siguiente sistema de ecuaciones diferenciales acopladas de segundo orden para las funciones $q_j(t)$:

$$ \ddot{q}_j - \sum_{i=1}^{N} f_{ij} q_j = 0 $$

Proponiendo soluciones de la forma:

$$ q_i(t) = A_i \cos(\omega^2 t + \phi) $$

y sustituyendo en el sistema de ecuaciones diferenciales se obtiene el conjunto de ecuaciones alegebráicas siguiente:

$$ \sum_{i=1}^{N} (f_{ij} -\delta_{ij} \omega^2) A_i = 0 $$

donde $\delta_{ij}$ es la delta de Kronecker que es igual a 1 si $i=j$ y cero si no. Este sistema de ecuaciones algebráicas es equivalente al problema de autovalores

$$ \mathbf{K} \mathbf{A} = \mathbf{\omega}^2 \mathbf{A} $$

donde $\mathbf{\omega}^2$ es una matriz diagonal.

La solución del probrema de autovalores da como resultado el conjunto de las frecuencias naturales del sistema (los autovalores) y el respectivo conjunto de desplazamientos para cada $\omega$ contenido en los vectores columna de la matriz $\mathbf{A}$.

La siguiente función calcula la matriz Hessiana del sistema:

In [ ]:
k_matrix = np.zeros((NPART,NPART))
for i in range(0,NPART):
    for j in range(0,NPART):
        k_matrix[i,j] = dd_energy(xcoords,i,j)

Aquí graficamos la matriz. Podemos ver que es (prácticamente) tridiagonal:

In [ ]:
plt.imshow(k_matrix)

En la siguiente sentencia se calculan los autovalores y autofunciones de la matriz Hessiana:

In [ ]:
evals, evecs = np.linalg.eigh(k_matrix)

En el siguiente gráfico se muestran los autovalores en orden creciente, salvo los dos modos de mayor frecuencia

In [ ]:
plt.plot(np.sqrt(evals[0:NPART-2]))

Aquí se amplía el detalle de los modos de baja frecuencia

In [ ]:
plt.plot(np.sqrt(evals),marker="o")
plt.xlim(0,6)
plt.ylim(0,9)

Los desplazamientos correspondientes a los cuatro modos de menor frecuencia se grafican aquí:

In [ ]:
plt.plot(evecs[:,0])
plt.plot(evecs[:,1])
plt.plot(evecs[:,2])
plt.plot(evecs[:,3])

Estas son las frecuencias y desplazamientos de los modos de mayor frecuencia

In [ ]:
print(np.sqrt(evals[NPART-2]))
print(np.sqrt(evals[NPART-1]))

In [ ]:
plt.plot(evecs[:,NPART-2])
plt.plot(evecs[:,NPART-1])

Estas son las frecuencias de los cuatro primeros modos:

In [ ]:
print(np.sqrt(evals[0]))
print(np.sqrt(evals[1]))
print(np.sqrt(evals[2]))
print(np.sqrt(evals[3]))

- ¿Cuál es la relación entre las frecuencias de los primeros modos?
- ¿Porqué esa relación es válida sólo para los modos de menor frecuencia?
- ¿Cómo depende la frecuencia del modo fundamental de la tensión de la cuerda?
- ¿A qué movimientos corresponden los dos modos de mayor frecuencia? ¿Porqué están ahí?